<a href="https://colab.research.google.com/github/kunal24bit/NLP/blob/main/Parts_of_speech_and_named_entity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Parts of Speech basics**

Most words are rare and its common for words that look completely different to mean almostthe same thing.
The same words in different order can mean something completely different.

Even spliiting text in to useful words like units can be difficult in many languages. While its possible to solve some problems starting from only the raw characters, its usually better to use linguistic knowledge to add useful information.

In [56]:
import spacy

In [57]:
nlp = spacy.load("en_core_web_sm")

In [58]:
doc = nlp(u"The quick brown fox jumped over the lazy dog's back")

In [59]:
print(doc.text)

The quick brown fox jumped over the lazy dog's back


In [60]:
print(doc[4].tag_)

VBD


In [61]:
print(doc[4].pos_)

VERB


In [62]:
for token in doc:
  print(f"{token.text:{10}} {token.tag_:{10}}{token.pos_:{10}} {spacy.explain(token.tag_)}")

The        DT        DET        determiner
quick      JJ        ADJ        adjective
brown      JJ        ADJ        adjective
fox        NN        NOUN       noun, singular or mass
jumped     VBD       VERB       verb, past tense
over       IN        ADP        conjunction, subordinating or preposition
the        DT        DET        determiner
lazy       JJ        ADJ        adjective
dog        NN        NOUN       noun, singular or mass
's         POS       PART       possessive ending
back       NN        NOUN       noun, singular or mass


Spacy is smart to differentiate between the past tense and present tense

In [66]:
doc1 = nlp(u"I read a book on NLP.")

In [67]:
word =  doc1[1]

token = word

token.text

'read'

In [68]:
print(f"{token.text:{10}} {token.tag_:{10}}{token.pos_:{10}} {spacy.explain(token.tag_)}")

read       VBD       VERB       verb, past tense


In [69]:
doc2 = nlp(u"I am reading a book on NLP")

In [70]:
word = doc2[2]

In [71]:
token = word
print(f"{token.text:{10}} {token.tag_:{10}}{token.pos_:{10}} {spacy.explain(token.tag_)}")

reading    VBG       VERB       verb, gerund or present participle


Counting Parts of Speech in DOC

In [79]:
POS_counts = doc.count_by(spacy.attrs.POS)

In [81]:
POS_counts

{84: 3, 85: 1, 90: 2, 92: 3, 94: 1, 100: 1}

In [84]:
for k,v in sorted(POS_counts.items()):
  print(f"{k}. {doc.vocab[k].text:{5}} {v}")

84. ADJ   3
85. ADP   1
90. DET   2
92. NOUN  3
94. PART  1
100. VERB  1


In [85]:
TAG_counts = doc.count_by(spacy.attrs.TAG)


In [92]:
for k,v in sorted(TAG_counts.items()):
  print(f"{k:{24}}. {doc.vocab[k].text:{10}} {v}")

                      74. POS        1
     1292078113972184607. IN         1
    10554686591937588953. JJ         3
    15267657372422890137. DT         2
    15308085513773655218. NN         3
    17109001835818727656. VBD        1


**Why did the ID numbers get so big?**

 In spaCy, certain text values are hardcoded into Doc.vocab and take up the first several hundred ID numbers. Strings like ‘NOUN’ and ‘VERB’ are used frequently by internal operations. Others, like fine-grained tags, are assigned hash values as needed.

**Visulizing parts of speech**

In [94]:
from spacy import displacy

In [107]:
displacy.render(doc, style='dep', jupyter = True, options={"distance":90})

**Named Entity Recognition**

NER seeks to locate and classify named entity mention in unstructured texts into pre defined categories such as the person names, organizations, locations, medical codes, time expressions, quatities, monetary values percentage etc.

Our goal is to read raw text such as:


*   Jim bought 300 shares of Acme corp. in 2006
*   [Jim]_person bought 300 shares of [Acme Corp]_Organization in [2006]_time.


In [115]:
def show_ents(doc):
  if doc.ents:
    for ent in doc.ents:
      print(ent.text +'  -  ' + ent.label_ + '  -  ' + str(spacy.explain(ent.label_)))
  else:
    print("No entities found")

In [116]:
doc = nlp(u"Hi, how are you?")

In [117]:
show_ents(doc)

No entities found


In [123]:
doc = nlp(u"may I got to Washington D.C. next may to see the Washington Monument")

In [124]:
show_ents(doc)

Washington D.C.  -  GPE  -  Countries, cities, states
next may  -  DATE  -  Absolute or relative dates or periods
Washington  -  GPE  -  Countries, cities, states


In [125]:
doc = nlp(u"Can I please have 500 dollars of Microsoft stock?")

In [126]:
show_ents(doc)

500 dollars  -  MONEY  -  Monetary values, including unit
Microsoft  -  ORG  -  Companies, agencies, institutions, etc.


In [141]:
doc = nlp(u"Tesla to build a U.K. factory for 500 million dolars")

In [142]:
show_ents(doc)

U.K.  -  GPE  -  Countries, cities, states
500 million  -  CARDINAL  -  Numerals that do not fall under another type


As we can see Tesla is not recognized as a company. 

In [143]:
from spacy.tokens import Span

In [144]:
ORG = doc.vocab.strings[u"ORG"]

In [145]:
ORG

383

In [146]:
new_ent = Span(doc, 0,1, label=ORG)

In [149]:
doc.ents = list(doc.ents) + [new_ent]

In [150]:
show_ents(doc)

Tesla  -  ORG  -  Companies, agencies, institutions, etc.
U.K.  -  GPE  -  Countries, cities, states
500 million  -  CARDINAL  -  Numerals that do not fall under another type


As we have added Tesla to be organization that is why it is giving Tesla -ORG